In [1]:
import numpy as np
import tensorflow as tf
import keras
from keras.models import load_model
import os
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.cluster import DBSCAN

from keras.optimizers import Adam

from cluster_classify_model import cluster_classify_model
from cluster_classify_util import *
from cluster_classify_train import *
from test_on_udacity_data import *

from keras.utils.generic_utils import get_custom_objects
get_custom_objects().update({"my_loss": my_loss})


Using TensorFlow backend.


In [2]:
# load trained model
model = load_model('./saved_model/test_cluster.h5')

[<tf.Tensor 'gradients/split_1_grad/concat:0' shape=(?, 7) dtype=float32>, None, None]


In [3]:
lidar_file1 = './data/training_didi_data/car_cluster/suburu_driving_towards_it/car_195.npy'
lidar_file2 = './data/training_didi_data/car_cluster/suburu_driving_towards_it/car_179.npy' #179

gtbox_file1 = './data/training_didi_data/car_train_gt_box_edited/suburu_driving_towards_it/gt_boxes3d/gt_boxes3d_195.npy' #179
gtbox_file2 = './data/training_didi_data/car_train_gt_box_edited/suburu_driving_towards_it/gt_boxes3d/gt_boxes3d_179.npy' #179


lidar1 = np.load(lidar_file1)
lidar2 = np.load(lidar_file2)
gtbox1 = np.load(gtbox_file1)
gtbox2 = np.load(gtbox_file2)

lidars = [lidar1, lidar2]
gtboxes = [gtbox1, gtbox2]

In [4]:
thresh = 0.5
nb_clusters = 2
centers = np.zeros((nb_clusters,2))
imgs = np.zeros((nb_clusters, 64,64,2))
for i in range(nb_clusters):
    img, center = discretize(lidars[1])
    imgs[i] = img
    centers[i] = center

features = model.predict(imgs)
thresh_features = features[features[:,0] >= thresh]
centers = centers[features[:,0] >= thresh]
boxes = np.array([gt_box_decode(features[i], centers[i], z_min = -1.5) for i in range(len(features)) ])

In [5]:
print(boxes.shape)

(2, 8, 3)


In [6]:
viz_mayavi_with_labels(lidars[1], gtboxes[1])

In [7]:
viz_mayavi_with_labels(lidars[1], np.expand_dims(boxes[1],0))

#### Predict on frame

In [8]:
def predict(model,lidar, seg_thres=0.5):
    lidar, labels = cluster(lidar)
    list_clusters = list(set(labels))
    nb_clusters = len(list_clusters)
    
    list_of_cluster = np.array([lidar[labels == list_clusters[i]] for i in range(nb_clusters)] )
    
    centers = np.zeros((nb_clusters,2))
    imgs = np.zeros((nb_clusters, 64,64,2))
    
    for i in range(nb_clusters):
        img, center = discretize(lidars[1])
        imgs[i] = img
        centers[i] = center

    features = model.predict(imgs)
    thresh_features = features[features[:,0] >= thresh]
    centers = centers[features[:,0] >= thresh]
    boxes = np.array([gt_box_decode(features[i], centers[i], z_min = -1.5) for i in range(len(features)) ])
    
    return boxes
    

In [9]:
lidar = np.load('./data/training_didi_data/car_train_edited/bmw_sitting_still/lidar/lidar_100.npy')
gtbox = np.load('./data/training_didi_data/car_train_gt_box_edited/bmw_sitting_still/gt_boxes3d/gt_boxes3d_100.npy')
viz_mayavi_with_labels(lidar, gtbox)

In [10]:
boxes = predict(model, lidar)
print(boxes.shape)

(1, 8, 3)


In [11]:
viz_mayavi_with_labels(lidar, boxes)